In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:12 http://ppa.launchpad.net/cran/libgit2/u

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-04-20 02:39:45--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2022-04-20 02:39:45 (6.56 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [68]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Lawn_and_Garden_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
raw_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Lawn_and_Garden_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
raw_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32787517| RED72VWWCOS7S|B008HDQYLQ|     348668413|Garden Weasel Gar...| Lawn and Garden|          1|            2|          8|   N|                Y|            One Star|I don't hate the ...| 2015-08-31|
|         US|   16374060| RZHWQ208LTEPV|B005OBZBD6|     264704759|10 Foot Mc4 Solar...| Lawn and Garden|          5|    

In [71]:
#Recreating the vine table 
vine_df = raw_df.select("review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase")
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RED72VWWCOS7S|          1|            2|          8|   N|                Y|
| RZHWQ208LTEPV|          5|            0|          0|   N|                Y|
|R37LBC3XAVLYOO|          5|            4|          5|   N|                Y|
|R3L7XJMA0MVJWC|          5|            0|          0|   N|                Y|
|R2I2GHSI7T1UBN|          1|            5|          6|   N|                Y|
|R2GFFKHK4I6VMX|          5|            0|          0|   N|                Y|
|R1R0UDX2XAN1S4|          4|            0|          0|   N|                Y|
|R22C8FMBSTFRY8|          5|            2|          2|   N|                Y|
|R118NNIQ75XPGO|          3|            0|          0|   N|                Y|
|R30HYXHZQ49621|          2|            0|          0|   N|     

In [74]:
#Filter the data 
#to retrieve all the rows where the total_votes count is equal to or greater than 20 to pick reviews 
#that are more likely to be helpful and to avoid having division by zero errors later on.
df_new_1 = vine_df.filter(vine_df.total_votes>= 20)
df_new_1.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQQ3KVTU5TJ4I|          5|           24|         29|   N|                Y|
|R3FELXWV9T5CWE|          2|           22|         24|   N|                Y|
| ROBYK6EZYK398|          5|           29|         30|   N|                Y|
|R2RKCSAG6GBA4A|          1|            8|         28|   N|                Y|
|R2YVBBR6NXIA4V|          5|           25|         28|   N|                N|
|R2AVTBDIVG2AW4|          5|           26|         26|   N|                N|
|R1Z2LNN3FANMTO|          1|           20|         24|   N|                N|
|  RLNULBKRWNNR|          5|           42|         43|   N|                Y|
| R9QNQUL94RX1F|          3|           27|         33|   N|                Y|
| RTULFZTUS1VBP|          5|           51|         52|   N|     

In [75]:
# Step - 2 - Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows where 
#            the number of helpful_votes divided by total_votes is equal to or greater than 50%.
df_new_2=df_new_1.filter((df_new_1.helpful_votes)/(df_new_1.total_votes)>=0.5)
df_new_2.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQQ3KVTU5TJ4I|          5|           24|         29|   N|                Y|
|R3FELXWV9T5CWE|          2|           22|         24|   N|                Y|
| ROBYK6EZYK398|          5|           29|         30|   N|                Y|
|R2YVBBR6NXIA4V|          5|           25|         28|   N|                N|
|R2AVTBDIVG2AW4|          5|           26|         26|   N|                N|
|R1Z2LNN3FANMTO|          1|           20|         24|   N|                N|
|  RLNULBKRWNNR|          5|           42|         43|   N|                Y|
| R9QNQUL94RX1F|          3|           27|         33|   N|                Y|
| RTULFZTUS1VBP|          5|           51|         52|   N|                Y|
|R1BM9RBQWI62O2|          5|           43|         60|   N|     

In [76]:
# Step 3 - Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that 
#           retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.
df_new_3_paid_review = df_new_2.filter(df_new_2.vine == 'Y')
df_new_3_paid_review.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R28DXTC3JQ9IY1|          4|           24|         26|   Y|                N|
|R3AFZKLQXATHBU|          5|           44|         49|   Y|                N|
|R2RUUF2JPJPC0E|          4|           20|         22|   Y|                N|
| RFZ2WUH4248AB|          2|           26|         27|   Y|                N|
|R1Q4LVHIFOWYFR|          5|           23|         28|   Y|                N|
| R4YEPTQED3X1Q|          5|           19|         20|   Y|                N|
|R2Z7C8YCRSC9DP|          5|           22|         22|   Y|                N|
|R3J8OI5CB74P5K|          1|           22|         25|   Y|                N|
| RH39LMKN6AZDC|          5|           33|         40|   Y|                N|
| R8RD8K0ESJSRD|          5|           21|         21|   Y|     

In [77]:
# Step - 4 - Repeat Step 3, but this time retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.
df_new_4_unpaid_review = df_new_2.filter(df_new_2.vine == 'N')
df_new_4_unpaid_review.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQQ3KVTU5TJ4I|          5|           24|         29|   N|                Y|
|R3FELXWV9T5CWE|          2|           22|         24|   N|                Y|
| ROBYK6EZYK398|          5|           29|         30|   N|                Y|
|R2YVBBR6NXIA4V|          5|           25|         28|   N|                N|
|R2AVTBDIVG2AW4|          5|           26|         26|   N|                N|
|R1Z2LNN3FANMTO|          1|           20|         24|   N|                N|
|  RLNULBKRWNNR|          5|           42|         43|   N|                Y|
| R9QNQUL94RX1F|          3|           27|         33|   N|                Y|
| RTULFZTUS1VBP|          5|           51|         52|   N|                Y|
|R1BM9RBQWI62O2|          5|           43|         60|   N|     

In [78]:
#Step - 5a.1 - Determine total number of reviews
total_reviews_count = vine_df.count()
total_reviews_count

2557288

In [79]:
#Step - 5a.2 - Determine total number of paid reviews (VINE REVIEWS)
total_paid_reviews_count = df_new_3_paid_review.count()
total_paid_reviews_count

386

In [80]:
#Step - 5a.3 - Determine total number of unpaid reviews (NOT VINE REVIEWS)
total_unpaid_reviews_count = df_new_4_unpaid_review.count()
total_unpaid_reviews_count

48717

In [81]:
#Step - 5b.1 - The number of 5-star reviews
count_total_fivestars = vine_df.filter(vine_df.star_rating == 5).count()
count_total_fivestars

1545893

In [82]:
#Step - 5b.2 - The number of 5-star paid reviews (VINE REVIEWS)
count_paid_fivestars = df_new_3_paid_review.filter(df_new_3_paid_review.star_rating == 5).count()
count_paid_fivestars

176

In [83]:
#Step - 5b.3 - The number of 5-star unpaid reviews (NOT VINE REVIEWS)
count_unpaid_fivestars = df_new_4_unpaid_review.filter(df_new_4_unpaid_review.star_rating == 5).count()
count_unpaid_fivestars

24026

In [84]:
total_pecentage=total_paid_reviews_count/total_paid_reviews_count*100
total_pecentage

100.0

In [85]:
#Step - 5c.1 - The percentage of 5-star paid reviews (VINE REVIEWS)
paid_pecentage=count_paid_fivestars/count_total_fivestars*100
paid_pecentage

0.011385005301143093

In [86]:
#Step - 5c.2 - The percentage of 5-star paid reviews (VINE REVIEWS)
unpaid_pecentage=count_unpaid_fivestars/count_total_fivestars*100
unpaid_pecentage

1.5541825986662725

In [87]:
#Summary dataframe 
dataframe = spark.createDataFrame([
                                   ("Total", total_reviews_count, count_total_fivestars, total_pecentage),
                                   ("Vine(Paid) ", total_paid_reviews_count, count_paid_fivestars, paid_pecentage),
                                   ("Non-Vine(Unpaid)", total_unpaid_reviews_count, count_unpaid_fivestars, unpaid_pecentage),
], ["Reviews", "Total Reviews", "Total 5-star Reviews", "Percentage of 5-star Reviews"])

dataframe.show()

+----------------+-------------+--------------------+----------------------------+
|         Reviews|Total Reviews|Total 5-star Reviews|Percentage of 5-star Reviews|
+----------------+-------------+--------------------+----------------------------+
|           Total|      2557288|             1545893|                       100.0|
|     Vine(Paid) |          386|                 176|        0.011385005301143093|
|Non-Vine(Unpaid)|        48717|               24026|          1.5541825986662725|
+----------------+-------------+--------------------+----------------------------+



In [88]:
# Convert PySpark DataFrame to Pandas DataFrame
import pandas as pd
pandas_df = dataframe.toPandas()
pandas_df

,Reviews,Total Reviews,Total 5-star Reviews,Percentage of 5-star Reviews
0,Total,2557288,1545893,100.000000
1,Vine(Paid),386,176,0.011385
2,Non-Vine(Unpaid),48717,24026,1.554183
